In [179]:
from pybedtools import BedTool
import sys
import pandas as pd

sys.argv=['test.opy','../../universal_data/rmsk/rmsk_GRCh38.txt','../../universal_data/ref/GRCh38/genes.bed']
rmsk=sys.argv[1]
gene_anno=sys.argv[2]

rmsk_df=pd.read_csv(rmsk,sep='\t')
rmsk_df['index']=rmsk_df[['repClass','repFamily','repName']].agg(':'.join,axis=1)

rmsk_bed=BedTool.from_dataframe(rmsk_df[['genoName','genoStart','genoEnd','index','swScore','strand']])
conn=rmsk_bed.intersect(anno_bed,s=True,wa=True,wb=True).to_dataframe()
conn[['ensg','gn']]=[x.split(':') for x in conn['blockCount']]
conn[['name','gn']].to_csv('../www/shiny-app/gene_te_net/network.txt',header=True,index=False)

   #bin  swScore  milliDiv  milliDel  milliIns genoName  genoStart   genoEnd  \
0     0     1892        83        59        14     chr1   67108753  67109046   
1     1     2582        27         0        23     chr1    8388315   8388618   
2     1     4085       171        77        36     chr1   25165803  25166380   
3     1     2285        91         0        13     chr1   33554185  33554483   
4     1     2451        64         3        26     chr1   41942894  41943205   

    genoLeft strand repName repClass repFamily  repStart  repEnd  repLeft  id  
0 -181847376      +    L1P5     LINE        L1      5301    5607     -544   1  
1 -240567804      -    AluY     SINE       Alu       -15     296        1   1  
2 -223790042      +   L1MB5     LINE        L1      5567    6174        0   4  
3 -215401939      -   AluSc     SINE       Alu        -6     303       10   6  
4 -207013217      -    AluY     SINE       Alu        -7     304        1   8  


,chrom,start,end,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStarts
0,chr1,8388315,8388618,SINE:Alu:AluY,2582,-,chr1,8352397,8848921,ENSG00000142599:RERE,1000,-
1,chr1,33554185,33554483,SINE:Alu:AluSc,2285,-,chr1,33513998,34165842,ENSG00000121904:CSMD2,1000,-
2,chr1,41942894,41943205,SINE:Alu:AluY,2451,-,chr1,41506365,42035925,ENSG00000127124:HIVEP3,1000,-
3,chr1,58719764,58720546,LINE:L2:L2a,1393,+,chr1,58715609,58771295,ENSG00000283445:ENSG00000283445,1000,+
4,chr1,150994812,150995102,SINE:Alu:AluSg,2212,+,chr1,150982249,150995634,ENSG00000143412:ANXA9,1000,+
